In [ ]:
import trimesh
import numpy as np
import tkinter as tk
from tkinter import filedialog, ttk, messagebox
import os
from PIL import Image, ImageTk
import io

# Optional GPU (CuPy)
try:
    import cupy as cp
    use_gpu = True
except ImportError:
    use_gpu = False
    cp = np

# === G-code Generation ===
def generate_gcode(mesh, layer_height, speed, extrusion_per_mm, output_path):
    z_layers = cp.arange(mesh.bounds[0][2], mesh.bounds[1][2], layer_height)
    gcode = [
        "G21 ; set units to mm",
        "G90 ; absolute positioning",
        "G28 ; home all axes",
    ]
    total_length = 0
    total_e = 0

    for z in z_layers:
        section = mesh.section(plane_origin=[0, 0, float(z)], plane_normal=[0, 0, 1])
        if section is None:
            continue
        paths, _ = section.to_planar()
        for entity in paths.entities:
            coords = entity.discrete(paths.vertices)
            coords = np.array(coords)
            if len(coords) < 2:
                continue
            gcode.append(f"G0 X{coords[0][0]:.2f} Y{coords[0][1]:.2f} Z{float(z):.2f}")
            for i in range(1, len(coords)):
                dx = coords[i][0] - coords[i-1][0]
                dy = coords[i][1] - coords[i-1][1]
                dist = np.sqrt(dx**2 + dy**2)
                e = dist * extrusion_per_mm
                gcode.append(f"G1 X{coords[i][0]:.2f} Y{coords[i][1]:.2f} Z{float(z):.2f} E{e:.4f} F{int(speed)}")
                total_length += dist
                total_e += e

    gcode += [
        "M104 S0 ; turn off extruder",
        "M140 S0 ; turn off bed",
        "G28 X0 ; home X",
        "M84 ; disable motors"
    ]

    with open(output_path, 'w') as f:
        f.write("\n".join(gcode))

    return len(z_layers), total_length, total_e

# === Render Preview Image ===
def render_preview(mesh):
    try:
        scene = mesh.scene()
        png = scene.save_image(resolution=(300, 300), visible=True)
        image = Image.open(io.BytesIO(png))
        return ImageTk.PhotoImage(image)
    except Exception as e:
        print("Preview render failed:", e)
        return None

# === GUI ===
class SlicerApp:
    def __init__(self, root):
        self.root = root
        self.root.title("🧠 Python 3D Slicer GUI")
        self.mesh = None

        self.file_label = tk.Label(root, text="No STL file loaded")
        self.file_label.pack(pady=5)

        tk.Button(root, text="Load STL", command=self.load_stl).pack(pady=5)

        self.preview_canvas = tk.Label(root)
        self.preview_canvas.pack(pady=10)

        self.layer_height = tk.DoubleVar(value=0.2)
        self.speed = tk.DoubleVar(value=1200)
        self.extrusion = tk.DoubleVar(value=0.05)
        self.layer_z = tk.DoubleVar(value=0.2)

        ttk.Label(root, text="Layer Height (mm)").pack()
        ttk.Entry(root, textvariable=self.layer_height).pack()

        ttk.Label(root, text="Speed (F)").pack()
        ttk.Entry(root, textvariable=self.speed).pack()

        ttk.Label(root, text="Extrusion per mm").pack()
        ttk.Entry(root, textvariable=self.extrusion).pack()

        ttk.Label(root, text="Preview Layer Z (mm)").pack()
        ttk.Entry(root, textvariable=self.layer_z).pack()

        tk.Button(root, text="Slice & Export G-code", command=self.slice).pack(pady=5)
        tk.Button(root, text="Show 3D View", command=self.show_3d).pack(pady=5)
        tk.Button(root, text="Preview Slice", command=self.preview_slice).pack(pady=5)

        self.status = tk.Label(root, text="")
        self.status.pack()

    def load_stl(self):
        path = filedialog.askopenfilename(filetypes=[("STL files", "*.stl")])
        if not path:
            return
        self.mesh = trimesh.load(path)
        if not isinstance(self.mesh, trimesh.Trimesh):
            self.mesh = self.mesh.dump().sum()
        self.file_label.config(text=f"Loaded: {os.path.basename(path)}")

        preview = render_preview(self.mesh)
        if preview:
            self.preview_canvas.config(image=preview)
            self.preview_canvas.image = preview

    def slice(self):
        if self.mesh is None:
            messagebox.showwarning("No Mesh", "Please load an STL file first.")
            return
        output_path = filedialog.asksaveasfilename(defaultextension=".gcode", filetypes=[("G-code", "*.gcode")])
        if not output_path:
            return
        self.status.config(text="⏳ Slicing...")
        self.root.update_idletasks()
        try:
            layers, path_len, total_e = generate_gcode(
                self.mesh,
                self.layer_height.get(),
                self.speed.get(),
                self.extrusion.get(),
                output_path
            )
            self.status.config(text=f"✅ Exported {layers} layers, {path_len:.1f} mm path, {total_e:.1f} E")
        except Exception as e:
            messagebox.showerror("Error", str(e))
            self.status.config(text="❌ Error during slicing")

    def show_3d(self):
        if self.mesh is None:
            messagebox.showwarning("No mesh", "Please load an STL file first.")
            return
        from vedo import Plotter, Mesh
        vmesh = Mesh(self.mesh)
        vp = Plotter(title="3D View", size=(800, 800))
        vp.show(vmesh, axes=1, viewup="z")

    def preview_slice(self):
        if self.mesh is None:
            messagebox.showwarning("No mesh", "Please load STL first.")
            return
        z = self.layer_z.get()
        section = self.mesh.section(plane_origin=[0, 0, z], plane_normal=[0, 0, 1])
        if section is None:
            messagebox.showinfo("No Slice", f"No slice found at Z = {z:.2f}")
            return
        paths, _ = section.to_planar()
        from vedo import Plotter, Lines
        lines = []
        for entity in paths.entities:
            coords = entity.discrete(paths.vertices)
            lines.append(Lines(coords))
        vp = Plotter(title=f"Slice at Z={z:.2f} mm", size=(600, 600))
        vp.show(*lines, axes=1)

# === Run App ===
if __name__ == '__main__':
    root = tk.Tk()
    app = SlicerApp(root)
    root.mainloop()


In [ ]:
pip install --upgrade -r requirements.txt